# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra
using ASEconvert

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]
    unit_cell = periodic_system(lattice, atoms, positions)

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE, make the supercell
    # and then convert back ...
    supercell_ase = convert_ase(unit_cell) * pytuple((repeat, 1, 1))
    supercell     = pyconvert(AbstractSystem, supercell_ase)

    # Unfortunately right now the conversion to ASE drops the pseudopotential information,
    # so we need to reattach it:
    supercell = attach_psp(supercell, Al="hgh/lda/al-q3")

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(supercell; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
convert_ase(periodic_system(setup.model)).write("al_supercell.png")

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123


Python None

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298384625965                   -0.85    5.4         
  2   -8.300179026843       -2.75       -1.25    1.0    134ms
  3   -8.300437048422       -3.59       -1.89    3.1    189ms
  4   -8.300461402410       -4.61       -2.75    1.6    150ms
  5   -8.300464022820       -5.58       -3.03    3.0    203ms
  6   -8.300464313777       -6.54       -3.20   12.1    394ms
  7   -8.300464467287       -6.81       -3.33    2.4    164ms
  8   -8.300464558064       -7.04       -3.48    1.5    151ms
  9   -8.300464623155       -7.19       -3.69    1.6    159ms
 10   -8.300464625164       -8.70       -3.73    1.0    129ms
 11   -8.300464638386       -7.88       -3.93    1.6    205ms
 12   -8.300464643868       -8.26      

In [4]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -16.64187689867                   -0.70    5.8         
  2   -16.67658258171       -1.46       -1.14    1.4    359ms
  3   -16.67920676083       -2.58       -1.88    2.1    384ms
  4   -16.67927810252       -4.15       -2.73    4.6    493ms
  5   -16.67928581591       -5.11       -3.10    4.4    510ms
  6   -16.67928620185       -6.41       -3.52    2.9    404ms
  7   -16.67928621764       -7.80       -4.01    2.2    364ms


In [5]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32299461008                   -0.56    5.8         
  2   -33.33194063433       -2.05       -1.01    1.0    1.28s
  3   -33.33398190202       -2.69       -1.76    5.2    1.83s
  4   -33.33462203267       -3.19       -2.40    3.2    1.56s
  5   -33.33692147559       -2.64       -2.62   10.2    2.39s
  6   -33.33693998499       -4.73       -3.05    3.5    1.43s
  7   -33.33694372626       -5.43       -3.80    3.6    1.64s
  8   -33.33694372748       -8.92       -4.11    4.4    1.86s


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298620926182                   -0.85    5.4         
  2   -8.300276619103       -2.78       -1.59    1.0    106ms
  3   -8.300429734995       -3.81       -2.58    1.9    122ms
  4   -8.300392489735   +   -4.43       -2.32    3.9    201ms
  5   -8.300464297298       -4.14       -3.38    1.1    109ms
  6   -8.300464591687       -6.53       -3.84    2.0    148ms
  7   -8.300464638053       -7.33       -4.20    1.4    153ms


In [7]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32655527992                   -0.56    6.9         
  2   -33.24356650040   +   -1.08       -1.23    1.2    1.11s
  3   +33.49067421537   +    1.82       -0.15    9.5    3.22s
  4   -33.31063231439        1.82       -1.66    6.1    2.72s
  5   -32.49138389083   +   -0.09       -1.06    3.8    2.00s
  6   -32.17123744188   +   -0.49       -1.01    4.8    1.98s
  7   -33.32368713006        0.06       -1.84    4.4    1.82s
  8   -33.31707022416   +   -2.18       -1.79    2.2    1.44s
  9   -33.33642025134       -1.71       -2.37    2.0    1.28s
 10   -33.33657865116       -3.80       -2.71    2.8    1.45s
 11   -33.33694069078       -3.44       -3.23    3.4    1.63s
 12   -33.33694222984       -5.81      

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.